In [1]:
#Create CSV of tweet column to send to GCS
to_predict = finaldf[['Tweet_nonstop']]

to_predict.to_csv('to_predict.csv')

In [ ]:
#Upload to_predict to GCS
bucket_name = "voorspel_bucket"
source_file_name = r"C:\Users\ensli\webanalytics\to_predict.csv"
destination_blob_name = "to_predict"

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""


    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        "File {} uploaded to {}.".format(
            source_file_name, destination_blob_name
        )
    )

upload_blob(bucket_name, source_file_name, destination_blob_name)

In [ ]:
file_path = "gs://voorspel_bucket/to_predict.csv"
model_name = "projects/alpine-alpha-300914/locations/us-central1/models/TST2983680932621320192"

def inline_text_payload(file_path):
    with open(file_path, 'rb') as ff:
        content = ff.read()
    return {'text_snippet': {'content': content, 'mime_type': 'text/plain'} }

def get_prediction(file_path, model_name):
    options = ClientOptions(api_endpoint='automl.googleapis.com')
    prediction_client = automl_v1.PredictionServiceClient(client_options=options)

    payload = inline_text_payload(file_path)
    
    params = {}
    request = prediction_client.predict(model_name, payload, params)
    return request  # waits until request is returned
    
if __name__ == '__main__':
    file_path = sys.argv[1]
    model_name = sys.argv[2]

    get_prediction(file_path, model_name)

In [ ]:
project_id = "alpine-alpha-300914"
model_id = "TST2983680932621320192"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="C:/Users/ensli/webanalytics/GCloud key/apikey.json"
from google.cloud import automl
content = 'wayfair is a bad company'
prediction_client = automl_v1.PredictionServiceClient()



# Get the full path of the model.
model_full_id = automl_v1.AutoMlClient.model_path(
    project_id, "us-central1", model_id
)


# Supported mime_types: 'text/plain', 'text/html'
# https://cloud.google.com/automl/docs/reference/rpc/google.cloud.automl.v1#textsnippet
text_snippet = automl_v1.TextSnippet(content=content, mime_type="text/plain")

payload = automl_v1.ExamplePayload(text_snippet=text_snippet)

print(payload)
response = prediction_client.predict(name=model_full_id, payload=payload)

print(response)

In [ ]:
model_name= "projects/alpine-alpha-300914/locations/us-central1/models/TST2983680932621320192"
content = 'SA to ban aclhocol sales'
project_id = "alpine-alpha-300914"
model_id = "TST2983680932621320192"
model_full_id = automl_v1.AutoMlClient.model_path(
    project_id, "us-central1", model_id
)


def get_prediction(content, model_full_id):
    options = ClientOptions(api_endpoint='automl.googleapis.com')
    prediction_client = automl_v1.PredictionServiceClient(client_options=options)
    
    text_snippet = automl_v1.TextSnippet(content=content, mime_type="text/plain")

    payload = automl_v1.ExamplePayload(text_snippet=text_snippet)
  # Uncomment the following line (and comment the above line) if want to predict on PDFs.
  # payload = pdf_payload(file_path)

    params = {}
    request = prediction_client.predict(name=model_full_id, payload=payload)
    return request  # waits until request is returned

get_prediction(content, model_full_id)

In [ ]:
model_full_id = automl_v1.AutoMlClient.model_path(project_id, "us-central1", model_id)

finaldf2['tweet_sentiment'] = finaldf2['Tweet_punct'].apply(lambda row: get_prediction(row, model_full_id))